# Importing all dependencies

In [34]:
import chardet
import pandas as pd
import numpy as np
import csv
from forex_python.converter import CurrencyRates

# Importing and connecting Arkouda

In [35]:
import arkouda as ak

In [36]:
ak.connect(connect_url='tcp://MacBook-Pro-7.local:5555 ') #connecting to arkouda server

/Users/narenkhatwani/Documents/GitHub/arkouda/arkouda/client.py:146: RuntimeWarning: Version mismatch between client (v2021.10.07+2.g1fd7670.dirty) and server (v2021.08.20+18.g7a657ad); this may cause some commands to fail or behave incorrectly! Updating arkouda is strongly recommended.
  warnings.warn(('Version mismatch between client ({}) and server ({}); ' +
connected to arkouda server tcp://*:5555


# 1. Read the Data

In [37]:
#Printing the first few values from the DataSet
!head /Users/narenkhatwani/Desktop/Resources/CSV_files/wfp_market_food_prices.csv

adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,1,2014,50.0,WFP
1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,2,2014,50.0,WFP
1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,3,2014,50.0,WFP
1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,4,2014,50.0,WFP
1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,5,2014,50.0,WFP
1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,6,2014,50.0,WFP
1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,7,2014,50.0,WFP
1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,8,2014,50.0,WFP
1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,9,2014,50.0,WFP


In [38]:
#Checking the encoding of the csv file, resolved the partial encoding error
with open("/Users/narenkhatwani/Desktop/Resources/CSV_files/wfp_market_food_prices.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

## Column Name Descriptions
1. adm0_id: country id
2. adm0_name: country name
3. adm1_id: locality id
4. adm1_name: locality name
5. mkt_id: market id
6. mkt_name: market name
7. cm_id: commodity purchase id
8. cm_name: commodity purchased
9. cur_id: currency id
10. cur_name: name of currency
11. pt_id: market type id
12. pt_name: market type (Retail/Wholesale/Producer/Farm Gate)
13. um_id: measurement id
14. um_name: unit of goods measurement
15. mp_month: month recorded
16. mpyear: year recorded *mpprice: price paid
17. mp_commoditysource: Source supplying price information

In [39]:
#Reading the csv file and creating a dataframe
df=pd.read_csv("/Users/narenkhatwani/Desktop/Resources/CSV_files/utf8/wfpvam_foodprices_utf8.csv")

#displaying the top 5 rows for testing
df.head(5)

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


#### In order to read the csv stating a specific encoding we can use the encoding parameter

In [40]:
#df=pd.read_csv("/Users/narenkhatwani/Documents/GitHub/arkouda/Notebooks/Resources/wfp_market_food_prices_utf8.csv,encoding='ascii')

# 2. Cleaning the Data 

## 2.1 Convert the csv to hdf file using to_hdf method of Python (not recommended)

In [41]:
df2=pd.read_csv("/Users/narenkhatwani/Desktop/Resources/CSV_files/utf8/wfpvam_foodprices_utf8.csv")

df2.to_hdf("/Users/narenkhatwani/Desktop/Resources/CSV_files/to_hdf/Food_prices_hdf5.h5", 'data', mode='w', format='table')


/Users/narenkhatwani/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [42]:
print(pd.read_hdf('/Users/narenkhatwani/Desktop/Resources/CSV_files/to_hdf/Food_prices_hdf5.h5', 'data').head(5))

   adm0_id    adm0_name  adm1_id   adm1_name  mkt_id  mkt_name  cm_id  \
0        1  Afghanistan      272  Badakhshan     266  Fayzabad     55   
1        1  Afghanistan      272  Badakhshan     266  Fayzabad     55   
2        1  Afghanistan      272  Badakhshan     266  Fayzabad     55   
3        1  Afghanistan      272  Badakhshan     266  Fayzabad     55   
4        1  Afghanistan      272  Badakhshan     266  Fayzabad     55   

          cm_name  cur_id cur_name  pt_id pt_name  um_id um_name  mp_month  \
0  Bread - Retail       0      AFN     15  Retail      5      KG         1   
1  Bread - Retail       0      AFN     15  Retail      5      KG         2   
2  Bread - Retail       0      AFN     15  Retail      5      KG         3   
3  Bread - Retail       0      AFN     15  Retail      5      KG         4   
4  Bread - Retail       0      AFN     15  Retail      5      KG         5   

   mp_year  mp_price  mp_commoditysource  
0     2014      50.0                 NaN  
1     

## 2.2 Conversion to .hdf5 using Pandas

In [43]:
pdgreen = pd.read_csv('/Users/narenkhatwani/Desktop/Resources/CSV_files/utf8/wfpvam_foodprices_utf8.csv')
# transfer columns of DataFrame to arkouda
def ak_create_akdict_from_df(df):
    akdict = {}
    for cname in df.keys():
        if df[cname].dtype.name == 'object':
            akdict[cname] = ak.from_series(df[cname],dtype=np.str)
        else:
            akdict[cname] = ak.from_series(df[cname])

    return akdict

/Users/narenkhatwani/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [44]:
#Passing the dataframe through the converter function to procure hdf file
green_from_pandas = ak_create_akdict_from_df(pdgreen)

In [45]:
print(green_from_pandas)

{'adm0_id': array([1 1 1 ... 152 152 152]), 'adm0_name': array(['Afghanistan', 'Afghanistan', 'Afghanistan', ... , 'Malawi', 'Malawi', 'Malawi']), 'adm1_id': array([272 272 272 ... 1890 1890 1890]), 'adm1_name': array(['Badakhshan', 'Badakhshan', 'Badakhshan', ... , 'Southern Region', 'Southern Region', 'Southern Region']), 'mkt_id': array([266 266 266 ... 823 823 823]), 'mkt_name': array(['Fayzabad', 'Fayzabad', 'Fayzabad', ... , 'Thondwe', 'Thondwe', 'Thondwe']), 'cm_id': array([55 55 55 ... 51 51 51]), 'cm_name': array(['Bread - Retail', 'Bread - Retail', 'Bread - Retail', ... , 'Maize - Retail', 'Maize - Retail', 'Maize - Retail']), 'cur_id': array([0 0 0 ... 0 0 0]), 'cur_name': array(['AFN', 'AFN', 'AFN', ... , 'MWK', 'MWK', 'MWK']), 'pt_id': array([15 15 15 ... 15 15 15]), 'pt_name': array(['Retail', 'Retail', 'Retail', ... , 'Retail', 'Retail', 'Retail']), 'um_id': array([5 5 5 ... 5 5 5]), 'um_name': array(['KG', 'KG', 'KG', ... , 'KG', 'KG', 'KG']), 'mp_month': array([1 2 3 .

In [46]:
data_pandas_hdf_convert= green_from_pandas

In [47]:
data_pandas_hdf_convert['adm0_id']

array([1 1 1 ... 152 152 152])

## 2.3 Conversion to .hdf5 using Arkouda

### Handling Currency Conversion

In [49]:
c = CurrencyRates()

In [50]:
a_dict=c.get_rates('USD')

In [51]:
a_dict.update(USD=1)

a_dict

{'EUR': 0.8643789437289308,
 'JPY': 113.89921341516121,
 'BGN': 1.6905523381450427,
 'CZK': 22.014867317832138,
 'DKK': 6.428991269772668,
 'GBP': 0.7377474284726424,
 'HUF': 311.055406690293,
 'PLN': 3.9819344800760654,
 'RON': 4.27815714409197,
 'SEK': 8.562537816578788,
 'CHF': 0.9122655372115134,
 'ISK': 129.82971734808538,
 'NOK': 8.521912006223529,
 'HRK': 6.5035871726164745,
 'RUB': 71.52243063358976,
 'TRY': 9.72452243063359,
 'AUD': 1.3486040280058778,
 'BRL': 5.604028005877776,
 'CAD': 1.2416803526666091,
 'CNY': 6.394934739389748,
 'HKD': 7.784510329328378,
 'IDR': 14364.871639726856,
 'INR': 74.41913734981415,
 'KRW': 1183.6200190163368,
 'MXN': 20.588728498573772,
 'MYR': 4.153513700406258,
 'NZD': 1.403319215143919,
 'PHP': 50.63445414469703,
 'SGD': 1.3507649753652,
 'THB': 33.31489324920045,
 'ZAR': 15.2040798686144,
 'USD': 1}

In [53]:
(pd.DataFrame.from_dict(data=a_dict, orient='index').to_csv('/Users/narenkhatwani/Desktop/Resources/CSV_files/currency_rates/dict_file.csv', header=True))


header = ["cur_name", "conversion_rate"]

with open('/Users/narenkhatwani/Desktop/Resources/CSV_files/currency_rates/dict_file.csv', 'r') as fp:
    reader = csv.DictReader(fp, fieldnames=header)

    # use newline='' to avoid adding new CR at end of line
    with open('/Users/narenkhatwani/Desktop/Resources/CSV_files/currency_rates/clean_rates.csv', 'w', newline='') as fh: 
        writer = csv.DictWriter(fh, fieldnames=reader.fieldnames)
        writer.writeheader()
        header_mapping = next(reader)
        writer.writerows(reader)#working

In [54]:
data1 = pd.read_csv('/Users/narenkhatwani/Desktop/Resources/CSV_files/utf8/wfpvam_foodprices_utf8.csv') 

data2 = pd.read_csv('/Users/narenkhatwani/Desktop/Resources/CSV_files/currency_rates/clean_rates.csv') 

# using merge function by setting how='left' 

output2 = pd.merge(data1, data2,on='cur_name',how='left') 

# displaying result 

In [58]:
output2.head(5) 

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource,conversion_rate
0,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0,AFN,15,Retail,5,KG,1,2014,50.0,NaN,NaN
1,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0,AFN,15,Retail,5,KG,2,2014,50.0,NaN,NaN
2,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0,AFN,15,Retail,5,KG,3,2014,50.0,NaN,NaN
3,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0,AFN,15,Retail,5,KG,4,2014,50.0,NaN,NaN
4,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0,AFN,15,Retail,5,KG,5,2014,50.0,NaN,NaN


In [59]:
output3 = output2[output2['conversion_rate'].notna()]

In [60]:
output3.head(5)

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource,conversion_rate
47965,115,Bassas da India,0,NaN,1887,National Average,52,Rice - Retail,0,INR,15,Retail,5,KG,7,2015,27.52,NaN,74.419137
47966,115,Bassas da India,0,NaN,1887,National Average,52,Rice - Retail,0,INR,15,Retail,5,KG,8,2015,27.34,NaN,74.419137
47967,115,Bassas da India,0,NaN,1887,National Average,52,Rice - Retail,0,INR,15,Retail,5,KG,9,2015,27.40,NaN,74.419137
47968,115,Bassas da India,0,NaN,1887,National Average,52,Rice - Retail,0,INR,15,Retail,5,KG,1,2016,27.06,NaN,74.419137
47969,115,Bassas da India,0,NaN,1887,National Average,52,Rice - Retail,0,INR,15,Retail,5,KG,2,2016,27.04,NaN,74.419137


In [61]:
output3.to_csv('/Users/narenkhatwani/Desktop/Resources/Final_csv_cleaned/global_food_clean.csv', index=False)

In [63]:
!python3 /Users/narenkhatwani/Documents/GitHub/arkouda/converter/csv2hdf.py \
--formats-file=/Users/narenkhatwani/Desktop/Resources/formatter_file/formatter_file.py\
--format=yellow \
--outdir=/Users/narenkhatwani/Desktop/Resources/Csv_final_pruned_merged\
/Users/narenkhatwani/Desktop/Resources/Final_csv_cleaned/global_food_clean.csv

Ignoring column adm0_name because dtype "object" has no HDF5 equivalent.
Ignoring column mkt_name because dtype "object" has no HDF5 equivalent.
Ignoring column cm_name because dtype "object" has no HDF5 equivalent.
Ignoring column cur_name because dtype "object" has no HDF5 equivalent.
Ignoring column pt_name because dtype "object" has no HDF5 equivalent.
Ignoring column um_name because dtype "object" has no HDF5 equivalent.
Columns to be extracted:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   adm0_id             1000 non-null   int64  
 1   adm1_id             1000 non-null   int64  
 2   adm1_name           0 non-null      float64
 3   mkt_id              1000 non-null   int64  
 4   cm_id               1000 non-null   int64  
 5   cur_id              1000 non-null   int64  
 6   pt_id               1000 non-null   int64 

In [64]:
green_from_HDF5 = ak.read_all('/Users/narenkhatwani/Desktop/Resources_arkouda/wfpvam_foodprices_utf8.hdf')

In [65]:
ak.get_datasets('/Users/narenkhatwani/Desktop/Resources_arkouda/wfpvam_foodprices_utf8.hdf')

['adm0_id',
 'adm1_id',
 'cm_id',
 'cur_id',
 'mkt_id',
 'mp_commoditysource',
 'mp_month',
 'mp_price',
 'mp_year',
 'pt_id',
 'um_id']

In [66]:
data=green_from_HDF5

In [67]:
data['adm0_id']

array([1 1 1 ... 152 152 152])

In [68]:
data['adm1_id']

array([272 272 272 ... 1890 1890 1890])

In [69]:
data['test_product']=data['adm0_id']*data['adm1_id']

In [70]:
data['test_product']

array([272 272 272 ... 287280 287280 287280])

## Arkouda Configuration

In [71]:
cfg=ak.get_config()

In [72]:
cfg

{'arkoudaVersion': 'v2021.08.20+18.g7a657ad',
 'ZMQVersion': '4.3.2',
 'HDF5Version': '1.12.1',
 'serverHostname': 'MacBook-Pro-7.local',
 'ServerPort': 5555,
 'numLocales': 1,
 'numPUs': 4,
 'maxTaskPar': 4,
 'physicalMemory': 8589934592,
 'distributionType': 'domain(1,int(64),false)',
 'LocaleConfigs': [{'id': 0,
   'name': 'MacBook-Pro-7.local',
   'numPUs': 4,
   'maxTaskPar': 4,
   'physicalMemory': 8589934592}],
 'authenticate': False,
 'logLevel': 'INFO',
 'byteorder': 'little'}